In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable
import torch.utils.data as data_utils
from torch.nn.modules import MSELoss, L1Loss, BCELoss

import glob
import csv
import cv2
from numpy import array, asarray, ndarray, swapaxes
from sklearn.preprocessing import MultiLabelBinarizer

In [2]:
#!pip install torchvision
#!pip install opencv-python

In [3]:
#training controls
batch_size = 20
epochs = 20
training_size = 0.7
learning_rate = 0.001
dropout = [0.3, 0.3, 0.3, 0.3, 0.2, 0.2, 0.2, 0.2, 0.15]

# input image dimensions
img_rows, img_cols = 268, 182

In [4]:
# the data holders
x_test = []
x_train = []
y_test= []
y_train= []
tempY = []

In [5]:
#opening the dataset
dataset = csv.reader(open("MovieGenre.csv",encoding="utf8",errors='replace'), delimiter=",")

#skipping the header line
next(dataset)

['\ufeffimdbId', 'Imdb Link', 'Title', 'IMDB Score', 'Genre', 'Poster']

In [6]:
import zipfile as zf
files = zf.ZipFile("SampleMoviePosters.zip", 'r')
files.extractall()
files.close()

In [7]:
#the list of image files in SampleMoviePosters folder
flist=glob.glob('SampleMoviePosters/*.jpg')  

In [8]:
len(flist)

997

In [9]:
image_ids = []

for path in flist:
    start = path.rfind("\\")+1
    end = len(path)-4
    image_ids.append(path[start:end])
    
#image_ids

In [10]:
import pandas as pd

dataset2 = pd.read_csv("MovieGenre.csv")
dataset2

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster
0,114709,http://www.imdb.com/title/tt114709,Toy Story (1995),8.3,Animation|Adventure|Comedy,https://images-na.ssl-images-amazon.com/images...
1,113497,http://www.imdb.com/title/tt113497,Jumanji (1995),6.9,Action|Adventure|Family,https://images-na.ssl-images-amazon.com/images...
2,113228,http://www.imdb.com/title/tt113228,Grumpier Old Men (1995),6.6,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...
3,114885,http://www.imdb.com/title/tt114885,Waiting to Exhale (1995),5.7,Comedy|Drama|Romance,https://images-na.ssl-images-amazon.com/images...
4,113041,http://www.imdb.com/title/tt113041,Father of the Bride Part II (1995),5.9,Comedy|Family|Romance,https://images-na.ssl-images-amazon.com/images...
...,...,...,...,...,...,...
40103,83168,http://www.imdb.com/title/tt83168,Tanya's Island (1980),4.3,Drama,https://images-na.ssl-images-amazon.com/images...
40104,82875,http://www.imdb.com/title/tt82875,Pacific Banana (1981),4.7,Comedy,https://images-na.ssl-images-amazon.com/images...
40105,815258,http://www.imdb.com/title/tt815258,Werewolf in a Womens Prison (2006),4.5,Horror,https://images-na.ssl-images-amazon.com/images...
40106,79142,http://www.imdb.com/title/tt79142,Xiao zi ming da (1979),6.5,Action|Comedy,https://images-na.ssl-images-amazon.com/images...


In [11]:
y = []
indexlist = []
classes = tuple()
ids = dataset2.imdbId.values.tolist()
for image_id in image_ids:
    genres = tuple((dataset2[dataset2["imdbId"] == int(image_id)]["Genre"].values[0]).split("|"))
    if int(image_id) in ids:
        indexlist.append(image_id)
    y.append(genres)
    classes = classes + genres
mlb = MultiLabelBinarizer()
mlb.fit(y)
y = mlb.transform(y)
classes = set(classes)
classes = list(classes)
classes.sort()

In [12]:
y_df = pd.DataFrame(y, columns = classes, index = indexlist)
y_df

,Action,Adventure,Animation,Biography,Comedy,Crime,Documentary,Drama,Family,Fantasy,...,Music,Musical,Mystery,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
10040,0,0,0,0,1,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
10057,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10071,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
10155,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
10195,0,1,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9899,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0
9932,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,1
9937,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0
9968,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,0,0,0,0,0


In [13]:
y_df_reset = y_df.reset_index()

y_df_reset["index"] = pd.to_numeric(y_df_reset["index"])

result = pd.merge(dataset2, y_df_reset, left_on='imdbId', right_on='index')

collist = result.columns.tolist()
collist.remove('Imdb Link')
collist.remove('IMDB Score')
collist.remove('index')
result = result[collist]

result

,imdbId,Title,Genre,Poster,Action,Adventure,Animation,Biography,Comedy,Crime,...,Music,Musical,Mystery,Romance,Sci-Fi,Short,Sport,Thriller,War,Western
0,24252,Liebelei (1933),Drama|Romance,https://images-na.ssl-images-amazon.com/images...,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
1,25316,It Happened One Night (1934),Comedy|Romance,https://images-na.ssl-images-amazon.com/images...,0,0,0,0,1,0,...,0,0,0,1,0,0,0,0,0,0
2,25164,The Gay Divorcee (1934),Comedy|Musical|Romance,https://images-na.ssl-images-amazon.com/images...,0,0,0,0,1,0,...,0,1,0,1,0,0,0,0,0,0
3,17350,The Scarlet Letter (1926),Drama,https://images-na.ssl-images-amazon.com/images...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,25586,Of Human Bondage (1934),Drama|Romance,https://images-na.ssl-images-amazon.com/images...,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1013,23797,Betty Boop's Big Boss (1933),Animation|Short|Comedy,https://images-na.ssl-images-amazon.com/images...,0,0,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1014,22670,Betty Boop's Museum (1932),Animation|Short|Comedy,https://images-na.ssl-images-amazon.com/images...,0,0,1,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1015,19640,Angora Love (1929),Comedy|Short,https://images-na.ssl-images-amazon.com/images...,0,0,0,0,1,0,...,0,0,0,0,0,1,0,0,0,0
1016,24676,TÌ«kyÌ« no onna (1933),Drama,https://images-na.ssl-images-amazon.com/images...,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
y_df_reset = y_df.reset_index()

shape = y_df_reset.shape[1]

index_value = []
genre_lst = []

for i in range(len(y_df_reset)):
    index_value.append(int(y_df_reset.loc[i,"index"]))
    temp_list = []
    for j in y_df_reset.columns[1:]:
        temp_list.append(y_df_reset.loc[i,j])
    genre_lst.append(temp_list)

df = pd.DataFrame(list(zip(index_value, genre_lst)),
               columns =['imdbId', 'genrelst'])

result = dataset2.merge(df, on="imdbId")
result

,imdbId,Imdb Link,Title,IMDB Score,Genre,Poster,genrelst
0,24252,http://www.imdb.com/title/tt24252,Liebelei (1933),7.7,Drama|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
1,25316,http://www.imdb.com/title/tt25316,It Happened One Night (1934),8.2,Comedy|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
2,25164,http://www.imdb.com/title/tt25164,The Gay Divorcee (1934),7.6,Comedy|Musical|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, ..."
3,17350,http://www.imdb.com/title/tt17350,The Scarlet Letter (1926),7.8,Drama,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
4,25586,http://www.imdb.com/title/tt25586,Of Human Bondage (1934),7.3,Drama|Romance,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."
...,...,...,...,...,...,...,...
1013,23797,http://www.imdb.com/title/tt23797,Betty Boop's Big Boss (1933),6.6,Animation|Short|Comedy,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1014,22670,http://www.imdb.com/title/tt22670,Betty Boop's Museum (1932),6.9,Animation|Short|Comedy,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1015,19640,http://www.imdb.com/title/tt19640,Angora Love (1929),7.0,Comedy|Short,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ..."
1016,24676,http://www.imdb.com/title/tt24676,TÌ«kyÌ« no onna (1933),7.2,Drama,https://images-na.ssl-images-amazon.com/images...,"[0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, ..."


In [15]:
#extracting the data from the CSV file
#for imdbId, Link, Title, Score, Genre, Poster in dataset2:
#    if(Score!=""):
#        if(len((int(imdbId),float(Score)))==2):
#            tempY.append((int(imdbId),float(Score))) # from Score to array of response binarys
for x in range(len(result)):
    tempY.append((int(result['imdbId'].iloc[x]),result['genrelst'].iloc[x]))

tempY

[(24252,
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
 (25316,
  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
 (25164,
  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]),
 (17350,
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 (25586,
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0]),
 (22879,
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]),
 (23969,
  [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0]),
 (22100,
  [0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0]),
 (13442,
  [0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]),
 (18578,
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0]),
 (19729,
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0]),
 (20629,
  [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0]),
 (21

In [16]:
#setting the length of training data
length=int(len(flist)*training_size)
length

697

In [17]:
#extracting the data about the images that are available
i=0
for filename in flist:
    name=int(filename.split('\\')[-1][:-4])
    for z in tempY:
        if(z[0]==name):
            
            img = array(cv2.imread(filename))
            img = swapaxes(img, 2,0)
            img = swapaxes(img, 2,1)

            if(i<length):
                x_train.append(img)
                y_train.append(z[1])
                i+=1
            else:
                x_test.append(img)
                y_test.append(z[1])
                i+=1

In [18]:
#converting the data from lists to numpy arrays
x_train=asarray(x_train,dtype=float)
x_test=asarray(x_test,dtype=float)
y_train=asarray(y_train,dtype=float)
y_test=asarray(y_test,dtype=float)

In [19]:
#scaling down the RGB data
x_train /= 255
x_test /= 255

In [20]:
#printing stats about the features
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

x_train shape: (697, 3, 268, 182)
697 train samples
321 test samples


In [21]:
train_length = x_train.shape[0]

x_train=torch.from_numpy(x_train)
x_test=torch.from_numpy(x_test)
y_train=torch.from_numpy(y_train)
y_test=torch.from_numpy(y_test)

train = data_utils.TensorDataset(x_train, y_train)
train_loader = data_utils.DataLoader(train, batch_size=batch_size, shuffle=False) # shuffle initially set to True

test = data_utils.TensorDataset(x_test, y_test)
test_loader = data_utils.DataLoader(test, batch_size=batch_size, shuffle=False)

In [22]:
# Sample Resnet model
from torchvision import models

class ResNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ResNet, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Linear(2048, len(classes))
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.resnet(x)
        x = self.sigmoid(x)
        return x

In [23]:
model = ResNet()
criterion = BCELoss()
optimizer = optim.Adam(model.parameters(), lr=learning_rate)

C:\Users\royal\anaconda3\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
C:\Users\royal\anaconda3\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [24]:
from statistics import mean

def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model(data)
        #print(output)
        #print(target)
        
        acc_list = []
        preds = torch.round(output)
        for i in range(len(preds)):
            result = 0
            for j in range(i):
                x = 0
                if preds[i][j] > 0.6:
                    x = 1
                if target[i][j] == x:
                    result+=1
            acc_list.append(result/len(classes))
                
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f}\tAccuracy: {:.2%}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), mean(acc_list)))

def test():
    print('test')
    model.eval()
    test_loss = 0
    correct = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        data, target = Variable(data, volatile=True).float(), Variable(target).float()
        output = model(data)
        
        acc_list = []
        preds = torch.round(output)
        for n in range(len(preds)):
            result = 0
            for m in range(n):
                if preds[n][m] == target[n][m]:
                    result+=1
            acc_list.append(result/len(classes))
            
        loss = criterion(output, target)
        test_loss += loss
        if loss==0:
            correct+=1

    print('\nTest set: \nAverage sq_loss: {:.4f} \nGuessed 100% correct: {:.4f}\nAccuracy: {:.2%}'.format(test_loss.data.item()/i, correct, mean(acc_list)))

model.float()
#print(model)
for epoch in range(0, epochs):
    train(epoch)
    test()

Train Epoch: 0 [0/697 (0%)]	Loss: 0.696641	Accuracy: 25.00%
Train Epoch: 0 [20/697 (3%)]	Loss: 0.399351	Accuracy: 38.70%
Train Epoch: 0 [40/697 (6%)]	Loss: 0.319402	Accuracy: 36.52%
Train Epoch: 0 [60/697 (9%)]	Loss: 0.261536	Accuracy: 37.83%
Train Epoch: 0 [80/697 (11%)]	Loss: 0.226974	Accuracy: 38.04%
Train Epoch: 0 [100/697 (14%)]	Loss: 0.316532	Accuracy: 35.00%
Train Epoch: 0 [120/697 (17%)]	Loss: 0.272308	Accuracy: 37.39%
Train Epoch: 0 [140/697 (20%)]	Loss: 0.245861	Accuracy: 35.87%
Train Epoch: 0 [160/697 (23%)]	Loss: 0.219746	Accuracy: 37.83%
Train Epoch: 0 [180/697 (26%)]	Loss: 0.258498	Accuracy: 36.09%
Train Epoch: 0 [200/697 (29%)]	Loss: 0.278180	Accuracy: 36.96%
Train Epoch: 0 [220/697 (31%)]	Loss: 0.288630	Accuracy: 36.52%
Train Epoch: 0 [240/697 (34%)]	Loss: 0.209101	Accuracy: 37.39%
Train Epoch: 0 [260/697 (37%)]	Loss: 0.233451	Accuracy: 38.70%
Train Epoch: 0 [280/697 (40%)]	Loss: 0.237693	Accuracy: 36.96%
Train Epoch: 0 [300/697 (43%)]	Loss: 0.239897	Accuracy: 38.04%
Tr

C:\Users\royal\AppData\Local\Temp\ipykernel_5424\3846039648.py:39: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
  data, target = Variable(data, volatile=True).float(), Variable(target).float()



Test set: 
Average sq_loss: 0.3189 
Guessed 100% correct: 0.0000
Accuracy: 0.00%
Train Epoch: 1 [0/697 (0%)]	Loss: 0.218009	Accuracy: 37.61%
Train Epoch: 1 [20/697 (3%)]	Loss: 0.262046	Accuracy: 37.39%
Train Epoch: 1 [40/697 (6%)]	Loss: 0.223174	Accuracy: 37.83%
Train Epoch: 1 [60/697 (9%)]	Loss: 0.231205	Accuracy: 36.96%
Train Epoch: 1 [80/697 (11%)]	Loss: 0.205514	Accuracy: 37.39%
Train Epoch: 1 [100/697 (14%)]	Loss: 0.224145	Accuracy: 38.91%
Train Epoch: 1 [120/697 (17%)]	Loss: 0.254085	Accuracy: 36.96%
Train Epoch: 1 [140/697 (20%)]	Loss: 0.212554	Accuracy: 37.83%
Train Epoch: 1 [160/697 (23%)]	Loss: 0.259881	Accuracy: 37.39%
Train Epoch: 1 [180/697 (26%)]	Loss: 0.305035	Accuracy: 35.22%
Train Epoch: 1 [200/697 (29%)]	Loss: 0.215550	Accuracy: 39.35%
Train Epoch: 1 [220/697 (31%)]	Loss: 0.198991	Accuracy: 36.74%
Train Epoch: 1 [240/697 (34%)]	Loss: 0.240518	Accuracy: 36.52%
Train Epoch: 1 [260/697 (37%)]	Loss: 0.228229	Accuracy: 37.61%
Train Epoch: 1 [280/697 (40%)]	Loss: 0.224368	

KeyboardInterrupt: 

In [ ]:
class Net(nn.Module):
    def __init__(self, input_shape=(3, img_rows, img_cols)):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(3, 128, kernel_size=3)
        self.conv1_drop = nn.Dropout2d(p=dropout[0])
        self.conv2 = nn.Conv2d(128, 64, kernel_size=3)
        self.conv2_drop = nn.Dropout2d(p=dropout[1])
        self.conv3 = nn.Conv2d(64, 64, kernel_size=3)
        self.conv3_drop = nn.Dropout2d(p=dropout[2])
        self.conv4 = nn.Conv2d(64, 64, kernel_size=2)
        self.conv4_drop = nn.Dropout2d(p=dropout[3])
        self.conv5 = nn.Conv2d(64, 32, kernel_size=2)
        self.conv5_drop = nn.Dropout2d(p=dropout[4])
        self.conv6 = nn.Conv2d(32, 16, kernel_size=2)
        self.conv6_drop = nn.Dropout2d(p=dropout[5])
        
        n_size = self._get_conv_output(input_shape)
        
        self.fc1 = nn.Linear(n_size, 16)
        self.fc1_drop = nn.Dropout(p=dropout[6])
        self.fc2 = nn.Linear(16, 16)
        self.fc2_drop = nn.Dropout(p=dropout[7])
        self.fc3 = nn.Linear(16, 8)
        self.fc3_drop = nn.Dropout(p=dropout[8])
        self.fc4 = nn.Linear(8, 1)
        
    def _get_conv_output(self, shape):
        bs = 1
        input = Variable(torch.rand(bs, *shape))
        output_feat = self._forward_features(input)
        n_size = output_feat.data.view(bs, -1).size(1)
        return n_size
        
    def _forward_features(self, x):
        x = F.relu(F.max_pool2d(self.conv1_drop(self.conv1(x)), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = F.relu(F.max_pool2d(self.conv3_drop(self.conv3(x)), 2))
        x = F.relu(F.max_pool2d(self.conv4_drop(self.conv4(x)), 2))
        x = F.relu(F.max_pool2d(self.conv5_drop(self.conv5(x)), 2))
        x = F.relu(F.max_pool2d(self.conv6_drop(self.conv6(x)), 2))
        return x
        
    def forward(self, x):
        x = F.relu(F.max_pool2d(self.conv1_drop(self.conv1(x)), 2))
        x = F.relu(F.max_pool2d(self.conv2_drop(self.conv2(x)), 2))
        x = F.relu(F.max_pool2d(self.conv3_drop(self.conv3(x)), 2))
        x = F.relu(F.max_pool2d(self.conv4_drop(self.conv4(x)), 2))
        x = F.relu(F.max_pool2d(self.conv5_drop(self.conv5(x)), 2))
        x = F.relu(F.max_pool2d(self.conv6_drop(self.conv6(x)), 2))
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1_drop(self.fc1(x)))
        x = F.relu(self.fc2_drop(self.fc2(x)))
        x = F.relu(self.fc3_drop(self.fc3(x)))
        x = self.fc4(x)
        return x


In [ ]:
model = Net()
criterion = MSELoss(size_average=True) #change to BinaryCrossEntropy loss
human_criterion = L1Loss(size_average=True)
optimizer = optim.RMSprop(model.parameters(), lr=learning_rate,
            alpha=0.9, eps=1e-08, weight_decay=0.0)

In [ ]:
def train(epoch):
    model.train()
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = Variable(data).float(), Variable(target).float()
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        loss.backward()
        optimizer.step()
        human_loss= human_criterion(output, target)
        print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss: {:.6f} {:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset),
            100. * batch_idx / len(train_loader), loss.data.item(), human_loss.data.item()))

def test():
    print('test')
    model.eval()
    test_loss = 0
    correct = 0
    human_loss = 0
    i = 0
    for batch_idx, (data, target) in enumerate(test_loader):
        i+=1
        data, target = Variable(data, volatile=True).float(), Variable(target).float()
        output = model(data)
        loss = criterion(output, target)
        test_loss += loss
        human_loss += human_criterion(output, target)
        if loss==0:
            correct+=1

    print('\nTest set: \nAverage sq_loss: {:.4f} \nAverage abs_loss: {:.4f} \nGuessed 100% correct: {:.4f}\n'.format(test_loss.data.item()/i, human_loss.data.item()/i, correct))

model.float()
#print(model)
for epoch in range(0, epochs):
    train(epoch)
    test()